In [31]:
import spacy
import os
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
# import gensim
# from gensim.utils import simple_preprocess
# import gensim.corpora as corpora
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from pprint import pprint





nlp = spacy.load("en_core_web_lg")  # make sure to use larger package!

## Path varibale defines from where the text files will be loaded
file_path = "all_papers"



In [32]:
def get_wv_sim(doc1,doc2):
    doc1 = nlp(doc1)
    doc2 = nlp(doc2)
    # Similarity of two documents
    return doc1.similarity(doc2)

def get_cosine_sim(doc1,doc2):
    X_list = word_tokenize(doc1)  
    Y_list = word_tokenize(doc2) 
    sw = stopwords.words('english')  
    l1 =[];l2 =[] 

    # remove stop words from the string 
    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
#     print("similarity: ", cosine)
    return cosine

def get_jaccard_sim(doc1, doc2): 
    a = set(doc1.split()) 
    b = set(doc2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [33]:
def get_primary_kw_sim(text):
    path = file_path+"/"
    filename = path+text
    doc1_f = open(filename,"r",encoding = "utf-8")
    doc2_f = open("primary-kw.txt","r",encoding = "utf-8")
    doc1 = doc1_f.read()
    doc2 = doc2_f.read()
    doc1 = clean_text(doc1)
    doc2 = clean_text(doc2)
    
    jaccard_sim = get_jaccard_sim(doc1,doc2)
    cosine_sim = get_cosine_sim(doc1,doc2)
    wv_sim = get_wv_sim(doc1,doc2)
    
    primary_writer.write(file+","+str(jaccard_sim)+","+str(cosine_sim)+","+str(wv_sim)+"\n")
    
    doc1_f.close()
    doc2_f.close()
    return jaccard_sim, cosine_sim,wv_sim


def get_secondary_kw_sim(text):
    path = file_path+"/"
    filename = path+text
    doc1_f = open(filename,"r",encoding = "utf-8")
    doc2_f = open("secondary-kw.txt","r",encoding = "utf-8")
    doc1 = doc1_f.read()
    doc2 = doc2_f.read()
    doc1 = clean_text(doc1)
    doc2 = clean_text(doc2)
    
    jaccard_sim = get_jaccard_sim(doc1,doc2)
    cosine_sim = get_cosine_sim(doc1,doc2)
    wv_sim = get_wv_sim(doc1,doc2)
    
    secondary_writer.write(file+","+str(jaccard_sim)+","+str(cosine_sim)+","+str(wv_sim)+"\n")
    
    doc1_f.close()
    doc2_f.close()
    return jaccard_sim, cosine_sim, wv_sim


def get_similarity(text):
    path = file_path+"/"
    filename = path+text
    doc1_f = open(filename,"r",encoding = "utf-8")
    docp_f = open("primary-kw.txt","r",encoding = "utf-8")
    docs_f = open("secondary-kw.txt","r",encoding = "utf-8")
    doc1 = doc1_f.read().lower()
    docp = docp_f.read().lower()
    docs = docs_f.read().lower()
    doc1 = clean_text(doc1)
    docp = clean_text(docp)
    docs = clean_text(docs)
    
    docp_ff = open("primary-kw.txt","r",encoding = "utf-8")
    docs_ff = open("secondary-kw.txt","r",encoding = "utf-8")
    
    pkw_list=docp_ff.readlines()
    pkw_list = [clean_text(x.lower().strip("\n")) for x in pkw_list ]
    skw_list=docs_ff.readlines()
    skw_list = [clean_text(x.lower().strip("\n")) for x in skw_list ]
    temp_pkw_list = []
    temp_skw_list = []
    not_found_pkw_list, not_found_skw_list = [], []
    
    for item in pkw_list:
#         item = item.strip("\n")
#         item = item.lower()
#         item = clean_text(item)
#         x = doc1.count(item)   ## For counting the occurance of the keyword
        x = doc1.find(item)
        if x != -1:
            temp_pkw_list.append(item)
#             doc_log.write(text+","+item+","+str(x)+","+"Primary"+"\n")
    for item in skw_list:
#         item = item.strip("\n")
#         item = item.lower()
#         item = clean_text(item)
#         x = doc1.count(item)   ## For counting the occurance of the keyword
        x = doc1.find(item)
        if x != -1:
            temp_skw_list.append(item)
#             doc_log.write(text+","+item+","+str(x)+","+"Secondary"+"\n")
            
    not_found_pkw_list = [x for x in pkw_list if x not in temp_pkw_list]
    not_found_skw_list = [x for x in skw_list if x not in temp_skw_list]
    
    doc_log.write(text+"\t"+str(temp_pkw_list)+"\t"+str(not_found_pkw_list)+"\t"+"Primary"+"\t"+str(len(temp_pkw_list))+"\t"+str(len(not_found_pkw_list))+"\t"+str(len(pkw_list))+"\n")
    doc_log.write(text+"\t"+str(temp_skw_list)+"\t"+str(not_found_skw_list)+"\t"+"Secondary"+"\t"+str(len(temp_skw_list))+"\t"+str(len(not_found_skw_list))+"\t"+str(len(skw_list))+"\n")
    
    jaccard_sim_p = get_jaccard_sim(doc1,docp)
    cosine_sim_p = get_cosine_sim(doc1,docp)
    wv_sim_p = get_wv_sim(doc1,docp)
    jaccard_sim_s = get_jaccard_sim(doc1,docs)
    cosine_sim_s = get_cosine_sim(doc1,docs)
    wv_sim_s = get_wv_sim(doc1,docs)
    
    sim_writer.write(text+","+str(jaccard_sim_p)+","+str(jaccard_sim_s)+","+str(cosine_sim_p)+","+str(cosine_sim_s)+","+str(wv_sim_p)+","+str(wv_sim_s)+"\n")
    
    doc1_f.close()
    docp_f.close()
    docs_f.close()
    
#     return jaccard_sim, cosine_sim, wv_sim

In [34]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     text = text.lower()
    text = re.sub(r"(\[[0-9].?\])|(\[[0-9].?.\])|(\[[0-9][0-9],[0-9][0-9]\])","",text)
    text = re.sub(r"\s{2,}"," ",text)
    text = re.sub(r"[A-Z]\s[A-Z]","",text)
    text = re.sub(r"(?<=[A-Z].)\s(?=[0-9])|(?<=[0-9])\s(?=[A-Z])|(?<=[A-Z])\s(?=[0-9])|(?<=\d)\s(?=\d)","",text)
    text = re.sub(r"\(","",text)
    text = re.sub(r"\)","",text)
    text = re.sub(r"\[","",text)
    text = re.sub(r"\]","",text)
    text = re.sub(r",","",text)
    text = re.sub(r";","",text)
    text = re.sub(r"~","",text)
    text = re.sub(r"'","",text)
    text = re.sub(r"\sÀ","-",text)
    text = re.sub(r"À","-",text)
    text = re.sub(r"Á","ση",text)
    text = re.sub(r"•","°",text)
    text = re.sub(r'[‘’“”…]', '', text)
    text = re.sub(r"Figure", '', text)
    return text

In [35]:
def get_filenames(file_extension):
    files = []
    for r,d, f in os.walk(file_path+"/"):
        for file in f:
            if "."+file_extension in file:
                files.append(file)
    return files

In [36]:
from datetime import datetime
# primary_writer = open("ALL_primary_kw_similarity.csv","a+",encoding="utf-8")
# secondary_writer = open("ALL_secondary_kw_similarity.csv","a+",encoding="utf-8")
sim_writer = open("ALL_kw_similarity_V2.csv","a+",encoding="utf-8")
sim_writer.write("Filename,Jaccard_primary,Jaccard_secondary,Cosine_primary,Cosine_secondary,Wordvector_primary,Wordvector_secondary\n")
doc_log = open("KW_list_found_not_found_V1.tsv","a+",encoding="utf-8")
doc_log.write("Filename \t Found Keywords \t Not Found Keywords \t KW Level \t Found Number \t Not Found Number \t Total Number \n")
# primary_writer.write("Filename,Jaccard,Cosine,Wordvector\n")
# secondary_writer.write("Filename,Jaccard,Cosine,Wordvector\n")


files_=get_filenames('txt')    

for file in files_:
    get_similarity(file)
    print(datetime.now().time(),": processed file: ",file,"\n") 
#     get_primary_kw_sim(file)
#     get_secondary_kw_sim(file)
    

    
# primary_writer.close()
# secondary_writer.close()
sim_writer.close()
doc_log.close()
print("Finished process at : ", datetime.now().time())

03:19:33.491378 : processed file:  1.tei.xml.txt 

03:19:35.386410 : processed file:  10.tei.xml.txt 

03:19:36.890379 : processed file:  3.tei.xml.txt 

03:19:38.541380 : processed file:  4.tei.xml.txt 

03:19:40.868382 : processed file:  5.tei.xml.txt 

03:19:42.476381 : processed file:  6.tei.xml.txt 

03:19:44.095409 : processed file:  7.tei.xml.txt 

03:19:45.491833 : processed file:  8.tei.xml.txt 

03:19:47.153804 : processed file:  9.tei.xml.txt 

03:19:48.512805 : processed file:  edlc101.tei.xml.txt 

03:19:49.455835 : processed file:  edlc105.tei.xml.txt 

03:19:50.821833 : processed file:  edlc106.tei.xml.txt 

03:19:52.425833 : processed file:  edlc108.tei.xml.txt 

03:19:54.122802 : processed file:  edlc109.tei.xml.txt 

03:19:55.195833 : processed file:  edlc110.tei.xml.txt 

03:19:56.520802 : processed file:  edlc111.tei.xml.txt 

03:19:58.408802 : processed file:  edlc112.tei.xml.txt 

03:19:59.964832 : processed file:  edlc12.tei.xml.txt 

03:20:01.452833 : processed 